In [2]:
from pathlib import Path
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [18]:
BASE_DIR = Path.cwd().parent

In [19]:
df_hosts_reviews_en_labeled = pd.read_parquet(BASE_DIR / 'processed_data' / 'hosts_reviews_en_labeled.parquet')

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

# Training data
from sklearn.model_selection import train_test_split
X = df_hosts_reviews_en_labeled['comments']
y = df_hosts_reviews_en_labeled['sentiment_label_class']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Count vectorizer to convert text into numerical features
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)




Accuracy: 0.9824145851277635


In [21]:
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix: \n', cm)

Confusion Matrix: 
 [[   42   212]
 [   33 13645]]


In [8]:
X_train

{'comments': ['hosts place unique charming old house ancient part city modern conveniences clean safe located steps away many attractions',
  'hosts flat great clean huge space bigger pictures nice sunny terraces even used laundry facility good wifi great location comfy bed communication easy prompt would recommend place anyone wants cool stay stockholm',
  'wonderful cocoon old stockholm perfect location close touristics places well back',
  'apartment nice quiet neighborhood far anything',
  'hosts place perfect stockholm one night place comfortable clean home conveniently located 5 mins away train station 10 mins away city center provided clean linen towels cute breakfast morning start day highly recommend youre stockholm',
  'absolutely dont recommend place especially room houseroom 150 years old needs major reconstruction unsafe unsuitable renting electricity went least 5 times avoid host asked us plug water kettle beside wc door boil water absolutely unsanitary electric plugs mak

In [6]:
 #Define the preprocessing steps for each column
text_preprocessing = Pipeline([
    ('countvectorizer', CountVectorizer()),
    ('tfidftransformer', TfidfTransformer())
])

price_preprocessing = Pipeline([
    ('standardscaler', StandardScaler())
])

# Combine the preprocessing steps using ColumnTransformer
preprocessor = ColumnTransformer([
    ('text', text_preprocessing, 'comments'),
    ('numerical', price_preprocessing, 'price')
])

# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', MultinomialNB())
])

# Fit the model to the training data
pipeline.fit(X_train, y_train)


ValueError: Expected 2D array, got scalar array instead:
array={'comments': ['nice convenient apartment short stay stockholm', 'host kind host made feel home even gave advice public transport nice park go walk place quiet slept well enough privacy room door directly kitchen thought convenient well', 'hostel expect hostel everything needed short stay beds comfy host really kind good recommendations', 'good place old town', 'great location clean comfortable also kind hosts definitely visit possible', 'recommend appartment clean good location really nice groups five host always ready answer questions thanks lot', 'hosts place family owned place personal touches helpful guide restaurants activities area really accessible area noisy noise around 10pm next door restaurant cleaning instrusive absolutely loved place back sure book days advanced booked day due accommodation falling cleaning fee price stay make sure get best price', 'great stay appartment instructions get clear easy follow appar

In [ ]:
# Evaluate the model on the testing data
y_pred = pipeline.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy:.3f}')

In [14]:
import pandas as pd
import numpy as np
import torch
import transformers
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Load and preprocess the data
df = df_hosts_reviews_en_labeled
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
X_text = df['comments'].values
X_price = df['price'].values.reshape(-1, 1)
X_price = StandardScaler().fit_transform(X_price)
X = {'input_ids': [], 'attention_mask': [], 'price': []}
for i in range(len(X_text)):
    encoded = tokenizer.encode_plus(X_text[i],
                                     add_special_tokens=True,
                                     max_length=128,
                                     padding='max_length',
                                     return_attention_mask=True,
                                     return_tensors='pt')
    if encoded['input_ids'].shape[1] != 128: # skip comments longer than max_length
        continue
    X['input_ids'].append(encoded['input_ids'])
    X['attention_mask'].append(encoded['attention_mask'])
    X['price'].append(X_price[i])
X['input_ids'] = torch.cat(X['input_ids'], dim=0)
X['attention_mask'] = torch.cat(X['attention_mask'], dim=0)
X['price'] = torch.tensor(np.concatenate(X['price'], axis=0), dtype=torch.float32)
y = torch.tensor(df['sentiment_label_class'].values, dtype=torch.long)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Fine-tune the BERT model
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=torch.utils.data.TensorDataset(X_train['input_ids'], X_train['attention_mask'], X_train['price'], y_train),
    eval_dataset=torch.utils.data.TensorDataset(X_test['input_ids'], X_test['attention_mask'], X_test['price'], y_test),
    data_collator=lambda data: {'input_ids': torch.stack([item[0] for item in data]),
                            'attention_mask': torch.stack([item[1] for item in data])})

ValueError: Found input variables with inconsistent numbers of samples: [3, 5000]

In [16]:
print(len(y))

5000


In [15]:
print(X['input_ids'].shape)


torch.Size([4976, 128])


In [ ]:
# Get predictions
predictions = trainer.predict(test_dataset=X_test)
predicted_classes = np.argmax(predictions.predictions, axis=1)

# Calculate accuracy
correct_predictions = predicted_classes == y_test.numpy()
accuracy = np.mean(correct_predictions)

# Calculate confusion matrix
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test.numpy(), predicted_classes)



In [ ]:
import pandas as pd
import numpy as np
import torch
import transformers
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, confusion_matrix

# Load and preprocess the data
df = df_hosts_reviews_en_labeled
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
X_text = df['comments'].values
X_price = df['price'].values.reshape(-1, 1)
X_price = StandardScaler().fit_transform(X_price)
X = {'input_ids': [], 'attention_mask': [], 'price': []}
for i in range(len(X_text)):
    encoded = tokenizer.encode_plus(X_text[i],
                                     add_special_tokens=True,
                                     max_length=128,
                                     padding='max_length',
                                     return_attention_mask=True,
                                     return_tensors='pt')
    X['input_ids'].append(encoded['input_ids'])
    X['attention_mask'].append(encoded['attention_mask'])
    X['price'].append(X_price[i])
X['input_ids'] = torch.cat(X['input_ids'], dim=0)
X['attention_mask'] = torch.cat(X['attention_mask'], dim=0)
X['price'] = torch.tensor(X['price'], dtype=torch.float32)
y = torch.tensor(df['sentiment_label_class'].values, dtype=torch.long)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define batch size
batch_size = 16

# Load the pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Fine-tune the BERT model
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=torch.utils.data.TensorDataset(X_train['input_ids'], X_train['attention_mask'], X_train['price'], y_train),
    eval_dataset=torch.utils.data.TensorDataset(X_test['input_ids'], X_test['attention_mask'], X_test['price'], y_test),
    data_collator=lambda data: {'input_ids': torch.stack([item[0] for item in data]),
                                'attention_mask': torch.stack([item[1] for item in data]),
                                'price': torch.stack([item[2] for item in data]),
                                'labels': torch.stack([item[3] for item in data])}
)
trainer.train()

# Get predictions and accuracy
y_pred = trainer.predict(test_dataset=torch.utils.data.TensorDataset(X_test['input_ids'], X_test['attention_mask'], X_test['price']))
y_pred = np.argmax(y_pred.predictions, axis=1)
acc = accuracy_score(y_test.numpy(), y_pred)

# Get confusion matrix
cm = confusion_matrix(y_test.numpy(), y_pred)
